## Prophet Demo
Just going to do a quick prophet demo using the transaction table from the dunnhumby data. I'm going to group all the baskets together, gather the time of each transaction and the sales, and project the total sales (as of writing this i dont know the time scale so it could just be for a week in advance, idk. I'm still just learning Prophet)

In [5]:
import pandas as pd
from fbprophet import Prophet

In [6]:
transaction_df = pd.read_csv('../Resources/dunnhumby/transaction_data.csv')

In [7]:
transaction_df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [8]:
sum_df = transaction_df.groupby('BASKET_ID').sum() 
## this is grouping by basket to get sales value for each transaction. 

In [9]:
# our next plan of attack is to merge the sum df with the transaction times. the way we grouped for
# sum_df, we took the sum of each category which works for sales value but not for time of day.
# since each basket_id is unique to each basket (duh), every row with the same basket_id *will*
# have identitcal TRANS_TIME and WEEK_NO attached to it, so doing the following line will work:
time_df = transaction_df.groupby('BASKET_ID').mean()

In [10]:
sum_df.head(2)

,household_key,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
BASKET_ID,,,,,,,,,,,
26984851472,11875,5,12316988,5,5.91,1820,-1.29,8155,5,0.0,0.0
26984851516,14250,6,19969639,7,12.43,2184,-3.26,9852,6,0.0,0.0


In [11]:
time_df.head(2)

,household_key,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
BASKET_ID,,,,,,,,,,,
26984851472,2375.0,1.0,2.463398e+06,1.000000,1.182000,364.0,-0.258000,1631.0,1.0,0.0,0.0
26984851516,2375.0,1.0,3.328273e+06,1.166667,2.071667,364.0,-0.543333,1642.0,1.0,0.0,0.0


In [12]:
# we will merge by index, which is the basket_id. but first I want to drop all the categories I don't need
sum_df = sum_df.drop(columns = ['household_key', 
                                'DAY', 
                                'PRODUCT_ID',
                                'QUANTITY',
                                'RETAIL_DISC',
                                'TRANS_TIME',
                                'WEEK_NO',
                                'COUPON_DISC',
                                'COUPON_MATCH_DISC'])
time_df = time_df.drop(columns = ['household_key', # the reason I didn't drop STORE_ID is because I may want to sort
                                  'PRODUCT_ID', # by store later, and I want to create a checkpoint right here.
                                  'QUANTITY', 
                                  'SALES_VALUE',
                                  'RETAIL_DISC',
                                  'COUPON_DISC', 
                                  'COUPON_MATCH_DISC'])

In [13]:
sum_df.head(2)

,SALES_VALUE,STORE_ID
BASKET_ID,,
26984851472,5.91,1820
26984851516,12.43,2184


In [14]:
time_df.head(2)

,DAY,STORE_ID,TRANS_TIME,WEEK_NO
BASKET_ID,,,,
26984851472,1.0,364.0,1631.0,1.0
26984851516,1.0,364.0,1642.0,1.0


## REVELATION
Unfortunately we need actual date time and dunnhumby did not provide such information